In [2]:
%serialconnect to port=/dev/tty.usbserial-0001 --baud=115200

Found serial ports: /dev/cu.usbserial-0001, /dev/cu.Bluetooth-Incoming-Port, /dev/cu.wlan-debug 
Connecting to --port=/dev/cu.usbserial-0001 --baud=115200 
Ready.


In [3]:
from machine import SoftI2C


mpu_addr = 0x68
mpu_pwr_mgmt = 0x6b

mpu_accel_x_h = 0x3b
mpu_accel_x_l = 0x3c
mpu_accel_y_h = 0x3d
mpu_accel_y_l = 0x3e
mpu_accel_z_h = 0x3f
mpu_accel_z_l = 0x40

mpu_gyro_x_h = 0x43
mpu_gyro_x_l = 0x44
mpu_gyro_y_h = 0x45
mpu_gyro_y_l = 0x46
mpu_gyro_z_h = 0x47
mpu_gyro_z_l = 0x48

mpu_gyro_sel = 340.0
mpu_accel_sel = 16348.0

class mpu6050():
    def __init__(self, i2c):
        self.i2c = i2c
        
    def power(self):
        self.i2c.writeto_mem(mpu_addr, mpu_pwr_mgmt, bytes([0]))

    def combine_register_values(self, h, l):
        if not h[0] & 0x80:
            return h[0] << 8 | l[0]
        return -((h[0] ^ 255) << 8) |  (l[0] ^ 255) + 1

    def get_accel(self):
        accel_x_h = self.i2c.readfrom_mem(mpu_addr, mpu_accel_x_h, 1)
        accel_x_l = self.i2c.readfrom_mem(mpu_addr, mpu_accel_x_l, 1)
        accel_y_h = self.i2c.readfrom_mem(mpu_addr, mpu_accel_y_h, 1)
        accel_y_l = self.i2c.readfrom_mem(mpu_addr, mpu_accel_y_l, 1)
        accel_z_h = self.i2c.readfrom_mem(mpu_addr, mpu_accel_z_h, 1)
        accel_z_l = self.i2c.readfrom_mem(mpu_addr, mpu_accel_z_l, 1)

        return [self.accel_sel(self.combine_register_values(accel_x_h, accel_x_l)),
                self.accel_sel(self.combine_register_values(accel_y_h, accel_y_l)),
                self.accel_sel(self.combine_register_values(accel_z_h, accel_z_l))]

    def accel_sel(self, accel):
        return accel / mpu_accel_sel

    def get_gyro(self):
        gyro_x_h = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_x_h, 1)
        gyro_x_l = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_x_l, 1)
        gyro_y_h = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_y_h, 1)
        gyro_y_l = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_y_l, 1)
        gyro_z_h = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_z_h, 1)
        gyro_z_l = self.i2c.readfrom_mem(mpu_addr, mpu_gyro_z_l, 1)

        return [self.gyro_sel(self.combine_register_values(gyro_x_h, gyro_x_l)),
                self.gyro_sel(self.combine_register_values(gyro_y_h, gyro_y_l)),
                self.gyro_sel(self.combine_register_values(gyro_z_h, gyro_z_l))]

    def gyro_sel(self, gyro):
        return gyro / mpu_gyro_sel


In [4]:
from machine import Pin, SoftI2C, UART
from time import sleep_ms
#from modules.sensor import mpu6050

if __name__ == "__main__":
    mpu = mpu6050(SoftI2C(scl=Pin(23), sda=Pin(22)))
    
    while True:
        print("Accelerometer:\t", mpu.get_accel(), "g")
        print("Gyroscope:\t", mpu.get_gyro(), "°/s")
        sleep_ms(1000)

Accelerometer:	 [0.3758258, -0.01517005, 1.049425] g
Gyroscope:	 [3.085294, -1.355882, -0.3470588] °/s
Accelerometer:	 [0.3922193, -0.01076584, 1.052117] g
Gyroscope:	 [3.026471, -1.25, -0.4176471] °/s
Accelerometer:	 [0.4010276, -0.01076584, 1.040617] g
Gyroscope:	 [2.964706, -0.9205883, -0.3411765] °/s
Accelerometer:	 [0.3885491, -0.01174456, 1.044776] g
Gyroscope:	 [2.970588, -1.220588, -0.3411765] °/s
Accelerometer:	 [0.3917299, -0.007340347, 1.052606] g
Gyroscope:	 [3.035294, -1.341177, -0.3705883] °/s
Accelerometer:	 [0.3929533, -0.01296795, 1.044042] g
Gyroscope:	 [3.064706, -1.135294, -0.4029412] °/s
Accelerometer:	 [0.3873257, -0.01027649, 1.05285] g
Gyroscope:	 [3.011765, -1.170588, -0.4323529] °/s
Accelerometer:	 [0.3954, 0.01517005, 1.048936] g
Gyroscope:	 [3.752941, -1.258824, -0.4441176] °/s
Accelerometer:	 [0.3865916, 0.01517005, 1.052606] g
Gyroscope:	 [3.002941, -1.144118, -0.3705883] °/s
Accelerometer:	 [0.3875704, 0.01443602, 1.047712] g
Gyroscope:	 [2.258824, -1.144

Traceback (most recent call last):
  File "<stdin>", line 11, in <module>
KeyboardInterrupt: 
